In [ ]:
import numpy as np
import xarray as xr
import pickle

In [ ]:
path = "/path/to/model/output/" 
eddypath = "/path/to/tracked/eddies/"

### Eddy masks
Here we create 4D (time, depth, lat, lon) masks that are 1 when an eddy is present at this location and 0 otherwise.

In [ ]:
for y in np.arange(201, 301):
    year = f"{y:04}"
    print("year", year)
    # loading the interpolated data without the extension
    data_int = xr.open_mfdataset(eddypath + 'interp_data.extend800.' + str(year) + '*.nc', 
                             concat_dim="time", combine="nested", data_vars='minimal', 
                             coords='minimal', compat='override').isel(lon=slice(0, 240))
    eddies_time_range = xr.cftime_range(start=year + '-01-03 00', end=year + '-12-30 00', 
                                        calendar='360_day', freq=str(5) + 'D')
    etr = [str(eddies_time_range[t]) for t in range(0, len(eddies_time_range))]
    k = 15
    # load the detected eddies
    eddies = {}
    for i in np.arange(0, len(etr)):
        datestring = etr[i][0:10]
        with open(eddypath + 'eddies/eddies_'
                  + datestring + '.pickle', 'rb') as f:
            eddies[i] = pickle.load(f)
        f.close()
    # the threshold of the OW paramter used for detection
    thresh = -0.3 * data_int.OW_std.mean("lon")
    tmpOW = data_int.OW
    # create dataset that is 1 whenever OW is < threshold
    ed_mask = tmpOW.where(tmpOW < thresh, 0)
    ed_mask = ed_mask.where(tmpOW >= thresh, 1)
    maske = np.zeros(np.shape(tmpOW))
    maske_cyc = np.zeros(np.shape(tmpOW))
    maske_ant = np.zeros(np.shape(tmpOW))
    for t in range(0, len(ed_mask.time)):
        emask = ed_mask.isel(time=t, z=0).values
        # set the locations where "OW < threshold" and "an eddy was detected" to one in the mask
        for e in range(0, len(eddies[t])):
            eddy_i = eddies[t][e]['eddy_i']
            eddy_j = eddies[t][e]['eddy_j']
            maske[t, 0, eddy_j, eddy_i] = emask[eddy_j, eddy_i]
            # do the same for cyclones and anticylones separately
            if eddies[t][e]['type'] == "cyclonic":
                maske_cyc[t, 0, eddy_j, eddy_i] = emask[eddy_j, eddy_i]
            elif eddies[t][e]['type'] == "anticyclonic":
                maske_ant[t, 0, eddy_j, eddy_i] = emask[eddy_j, eddy_i]
    # and write the masks to disk
    em = xr.DataArray(maske, coords={"time": ed_mask.time, "z": ed_mask.z,
                                     "lat": ed_mask.lat, "lon": ed_mask.lon},
                          dims=["time", "z", "lat", "lon"], name='eddymask')
    em.to_netcdf(eddypath + "eddymask_" + str(year) + ".nc")
    em_cyc = xr.DataArray(maske_cyc, coords={"time": ed_mask.time, "z": ed_mask.z,
                                                 "lat": ed_mask.lat, "lon": ed_mask.lon},
                              dims=["time", "z", "lat", "lon"], name='eddymask_cyclones')
    em_cyc.to_netcdf(eddypath + "eddymask_cyclones_" + str(year) + ".nc")
    em_ant = xr.DataArray(maske_ant, coords={"time": ed_mask.time, "z": ed_mask.z,
                                                 "lat": ed_mask.lat, "lon": ed_mask.lon},
                              dims=["time", "z", "lat", "lon"], name='eddymask_anticyclones')
    em_ant.to_netcdf(eddypath + "eddymask_anticyclones_" + str(year) + ".nc")